<h1>Содержание<span class="tocSkip"></span></h1>
</span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Токенизация-и-лемматизация" data-toc-modified-id="Токенизация-и-лемматизация-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Токенизация и лемматизация</a></span></li><li><span><a href="#Деление-выборки:-тестовая-и-валидационная" data-toc-modified-id="Деление-выборки:-тестовая-и-валидационная-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Деление выборки: тестовая и валидационная</a></span></li><li><span><a href="#Векторизация-тестовой-и-валидационной-выборки" data-toc-modified-id="Векторизация-тестовой-и-валидационной-выборки-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Векторизация тестовой и валидационной выборки</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Кросс-валидация-RandomizedSearchCV-для-LGBMRegressor" data-toc-modified-id="Кросс-валидация-RandomizedSearchCV-для-LGBMRegressor-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Кросс-валидация RandomizedSearchCV для LGBMRegressor</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить несколько моделей. 
3. Сформулировать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [29]:
import pandas as pd                                         # библиотека pandas
import numpy as np                                          # библиотека работы с массивами numpy
import nltk                                                 # платформа для работы с данными человеческого языка
import re                                                   # встроенный модуль для работы с регулярными выражениями
from nltk.corpus import stopwords as nltk_stops             # загрузка стоп слов не несущих смысловой нагрузки               
from nltk.stem import WordNetLemmatizer                     # иснтрумент лемматизации (английского языка)     
nltk.download('wordnet')

from sklearn.linear_model import LogisticRegression         # алгоритм — логистическая регрессия
from sklearn.feature_extraction.text import TfidfVectorizer # класс TfidfVectorizer, счётчик упоминания слов для создания векторов
from sklearn.model_selection import train_test_split        # иструмент деления выборки
from sklearn.model_selection import RandomizedSearchCV      # импортируем инструмент кросс-валидации и обучения модели (рандом)
from lightgbm import LGBMClassifier                         # алгоритм «лёгкого градиентного бустинга»
from scipy.stats import randint                             # подбор рандомных значений в диапазоне

from sklearn.metrics import f1_score                        #f1 мера (среднее гармоническое полноты и точности)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
# оценим балан классов по токсичности

display(df.groupby('toxic').count())

,text
toxic,
0,143346
1,16225


In [8]:
print("Отношение позитивных комментариев к негативным:",
      round(df.loc[df['toxic'] == 0, 'toxic'].count() / df.loc[df['toxic'] == 1, 'toxic'].count(),2))

Отношение позитивных комментариев к негативным: 8.83


**Вывод**: наблюдается дисбаланс классов, токсичных комментариев в 9 раз меньше, чем не токсичных.

## Обучение

Cхема:
1. Проводится предобработка текста:
    - Выполняют токенизацию каждого текста, то есть его разбивают на слова;
    - Слова лемматизируют: приводят к начальной словарной форме;
    - Текст очищают от стоп-слов и ненужных символов;
    - Для корректной работы алгоритма добавляют маркеры начала и конца предложения (они приравниваются к токенам).
2. На выходе у каждого исходного текста образуется свой список токенов.
3. Затем токены передают модели, которая переводит их в векторные представления.
    - На выходе для каждого текста образуются векторы заданной длины.
4. На финальном этапе модели передают признаки (векторы) и она прогнозирует эмоциональную окраску текста.

In [9]:
corpus = df['text'].values.astype('U') # переводим текс в корпус / лист слов 

In [10]:
corpus[0:3]

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."],
      dtype='<U5000')

### Токенизация и лемматизация

In [11]:
# функция перевода текста в лемматизированную форму:

def lemma(text):
    
    lemmatizer = WordNetLemmatizer()
    
    token_list = nltk.word_tokenize(text)                                         # токенезируем текст, разделим на слова
      
    lemm_text = " ".join([lemmatizer.lemmatize(w).lower() for w in token_list])   # лемматизируем и собираем текст в одну строку
    
    text_clear = " ".join(re.sub(r'[0-9 ]', ' ', lemm_text).split())
    
    return text_clear

In [13]:
for i in range(len(corpus)):                  # преобразуем каждую строку с помощью формулы
    corpus[i] = lemma(corpus[i])

In [14]:
corpus[0:3]

array(["explanation why the edits made under my username hardcore metallica fan were reverted ? they were n't vandalism , just closure on some gas after i voted at new york dolls fac . and please do n't remove the template from the talk page since i 'm retired now. . . .",
       "d'aww ! he match this background colour i 'm seemingly stuck with . thanks . ( talk ) : , january , ( utc )",
       "hey man , i 'm really not trying to edit war . it 's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page . he seems to care more about the formatting than the actual info ."],
      dtype='<U5000')

### Деление выборки: тестовая и валидационная

In [15]:
# разделим корпус текстов не перемешивая, с установленным random_state для повторяемости

train_corpus, valid_corpus = train_test_split(corpus, random_state = 42, test_size = 0.25, shuffle = False)

In [16]:
# разделим целевой приззнак не перемешивая, с установленным random_state для повторяемости

train_target, valid_target = train_test_split(df['toxic'], random_state = 42, test_size = 0.25, shuffle = False)

### Векторизация тестовой и валидационной выборки

In [17]:
nltk.download('stopwords')                               # загрузка модуля стоп-слова

stops = set(nltk_stops.words('english'))                 # загрузка стоп слов на английском языке

count_tf_idf = TfidfVectorizer(stop_words=stops)         # модуль векторизации корпуса текстов

tf_idf_train = count_tf_idf.fit_transform(train_corpus)  # обучение и векторизация обучающей выборки

tf_idf_valid = count_tf_idf.transform(valid_corpus)      # векторизация валидационной выборки без обучения

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Обучение моделей

#### Логистическая регрессия

In [19]:
reg_model = LogisticRegression(solver='lbfgs', random_state=42, max_iter=1500)

reg_model.fit(tf_idf_train, train_target)

predictions_reg = reg_model.predict(tf_idf_valid)
    
result_log_reg = f1_score(valid_target, predictions_reg)

In [22]:
print("Результат метрики F1 для логистической регрессии:", round(result_log_reg,5))

Результат метрики F1 для логистической регрессии: 0.72555


**Вывод**: Результат метрики ниже целевого показателя равного 0.75, продолжаем поиск модели. 

#### Кросс-валидация RandomizedSearchCV для LGBMRegressor

In [31]:
param_grid = {'n_estimators': randint(100, 300),          # количество деревьев в модели
              'learning_rate': np.linspace(0.1, 0.5, 5),   # скорость обучения модели / скорость сокращения шага градиентного спуска
              'max_depth': randint(5, 15),                 # глубина дерева в модели
              'num_leaves': randint(2, 15) 
             }
random_search_lgbm = RandomizedSearchCV(
                                   LGBMClassifier(random_state=42,verbose=0),
                                   param_grid,
                                   scoring='f1',
                                   random_state=42,
                                   cv=5,
                                   n_jobs=-1
                                  )

random_search_lgbm.fit(tf_idf_train, train_target)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(random_state=42, verbose=0),
                   n_jobs=-1,
                   param_distributions={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5]),
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f353fabcb50>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f353faa3590>,
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f353fabc790>},
                   random_state=42, scoring='f1')

In [32]:
print("Результат по f1 для LGBM", round(random_search_lgbm.best_score_,4))
print()
print("Параметры модели:", random_search_lgbm.best_params_)

Результат по f1 для LGBM 0.7621

Параметры модели: {'learning_rate': 0.4, 'max_depth': 10, 'n_estimators': 229, 'num_leaves': 13}


**Вывод:** Результаты подбора параметров для модели "Легкого градиентного бустинга" позволили получить метрику, удовлетворяющую требованию задания. Параметры модели 'learning_rate': 0.4, 'max_depth': 10, 'n_estimators': 229, num_leaves': 13. Протестируем полученные результаты на валидационной выборке.

In [34]:
best_lgbm_model = LGBMClassifier(learning_rate = 0.4, max_depth = 10, n_estimators = 229, num_leaves = 13).fit(tf_idf_train, train_target)

predictions_lgbm = best_lgbm_model.predict(tf_idf_valid)

result_lgbm = f1_score(valid_target, predictions_lgbm)

print("Результат метрики F1 для LGBM на валидационной выборке:", round(result_lgbm,5))

Результат метрики F1 для LGBM на валидационной выборке: 0.7627


## Выводы

По результатам работы были проделаны следующие шаги:

1. Подготовка текста к машинному обучению:
  - токенизация, лемматизация;
  - убраны "лишние" цифровые символы в данных;

2. Осуществлено деление выборки на обучающую и валидационную.

3. Осуществлена векторизация (TF-IDF) для обучающей выборки и валидационной.
  - обучение векторизации не проводилось для валидационной выборке, только трансформация по результатам тренировки на обучающей.

4. Проведено обучение моделей "логистической регрессии" и "легкого градиентного бустинга". Наилучшие результаты получены для бустинга: параметры модели 'learning_rate': 0.4, 'max_depth': 10, 'n_estimators': 229, num_leaves': 13.

5. Полученный результат метрики f1 равен 0.76.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны